1. Попробуйте изменить параметры нейронной сети работающей с датасетом imdb либо нейронной сети работающей airline-passengers(она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить ее точность. Приложите анализ.

In [19]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation
from keras.layers import LSTM
from keras.datasets import imdb

import numpy as np
from keras.layers.recurrent import SimpleRNN, LSTM, GRU


In [14]:
max_features = 20000
# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 90
batch_size = 20 # увеличьте значение для ускорения обучения

In [15]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности


In [16]:
print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 90)
x_test shape: (25000, 90)


In [17]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Построение модели...


In [18]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=2, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Процесс обучения...


D:\Prog\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 133s 5ms/step - loss: 0.4521 - accuracy: 0.7884 - val_loss: 0.4190 - val_accuracy: 0.8088
Epoch 2/2
25000/25000 [==============================] - 23s 901us/step
Результат при тестировании: 0.3909497666180134
Тестовая точность: 0.8357599973678589


Оригинальные параметры
Результат при тестировании: 0.39250763788819315
Тестовая точность: 0.8256000280380249

Изменены batch size (20), maxlen (90), epochs=3
Результат при тестировании: 0.37212768812179564
Тестовая точность: 0.8463199734687805

Изменены epochs=2
Результат при тестировании: 0.3909497666180134
Тестовая точность: 0.8357599973678589

Очевиднее всего влияет увеличение эпох

2. Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения

In [21]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

In [22]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [23]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [30]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 256
NUM_ITERATIONS = 50 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [31]:

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


In [32]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/1
143504/143504 [==============================] - 46s 317us/step - loss: 2.5250
Генерация из посева: l just exp
l just expere the said the said the said the said the said the said the said the said the said the said the sa==================================================
Итерация #: 1
Epoch 1/1
143504/143504 [==============================] - 48s 331us/step - loss: 2.0454
Генерация из посева: before he 
before he the was the was the was the was the was the was the was the was the was the was the was the was the ==================================================
Итерация #: 2
Epoch 1/1
143504/143504 [==============================] - 48s 331us/step - loss: 1.8421
Генерация из посева: uously.  `
uously.  `i dould the gryphon and and the gryphon and and the gryphon and and the gryphon and and the gryphon ==================================================
Итерация #: 3
Epoch 1/1
143504/143504 [==============================] - 48s 335us/step - loss: 1.7041
Генерация

alice waited to see that she was to sey a whiting to its feet, `i might as well say,' added the gryphon went o==================================================
Итерация #: 28
Epoch 1/1
143504/143504 [==============================] - 48s 332us/step - loss: 0.5950
Генерация из посева: rd-mine ne
rd-mine near her face, and was in the schoolroom, and repeat semied there was nothing of the stack. `they're d==================================================
Итерация #: 29
Epoch 1/1
143504/143504 [==============================] - 47s 328us/step - loss: 0.5702
Генерация из посева: quite jump
quite jumped up and bawled out, `sit down, wo line to see the hatter went on, `you throw them she her furrowin==================================================
Итерация #: 30
Epoch 1/1
143504/143504 [==============================] - 47s 330us/step - loss: 0.5482
Генерация из посева:  down that
 down that makes them so shiny she sat down and looked along the baby at her as she spoke. alice had been bor

it is wrong for a moment to think this a good opportunity of saying to herself, i don't think

Были изменены BATCH_SIZE, HIDDEN_SIZE = 256, 256. Кол-во итераций поднято до 50